# Fine-Tuning Llama 3.2 1B (Huggingface)
- Single GPU Version

In [1]:
import warnings
warnings.filterwarnings(action='ignore') # 경고 메시지를 무시하고 숨기거나

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
hf_token = os.environ.get('HF_TOKEN',"")

In [4]:
# !pip install -U peft trl transformers diffusers
# !pip install bitsandbytes

In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

2025-04-14 17:16:48.326654: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-14 17:16:48.342650: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-14 17:16:48.347635: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 17:16:48.360003: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:

MODEL_NAME = "meta-llama/Llama-3.2-1B"  # Llama 3.2 1B 모델 ID
# MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"  # Llama 3.2 1B 모델 ID
OUTPUT_DIR = "./llama-3.2-1b-finetuned"
# DATASET_PATH = "train.jsonl"  # 또는 JSONL 파일 경로
DATASET_PATH = "dataset/kogpt/mini_ai_dataset.jsonl"  # 또는 JSONL 파일 경로

In [7]:


# 2. 학습 파라미터 설정
max_seq_length = 512

training_params = {
    "learning_rate": 2e-5,
    # "max_seq_length": 512,
    "num_train_epochs": 10,
    "per_device_train_batch_size": 1, #<--- 배치를 높이면, Memroy 부족으로 OOM이 발생합니다. 
    "gradient_accumulation_steps": 4,
    "optim": "adamw_torch",
    "save_strategy": "epoch",
    "logging_steps": 100,
    "eval_steps": 500,
    "warmup_ratio": 0.1,
    "lr_scheduler_type": "cosine",
    "bf16": True,  # BF16 사용 (A100, H100 등 최신 GPU에서 지원)
}

In [8]:
# 3. 데이터셋 로드
# CSV 파일인 경우
if DATASET_PATH.endswith('.csv'):
    dataset = load_dataset('csv', data_files=DATASET_PATH)
# JSONL 파일인 경우
elif DATASET_PATH.endswith('.jsonl'):
    dataset = load_dataset('json', data_files=DATASET_PATH, split='train')
else:
    raise ValueError("지원되지 않는 파일 형식입니다. CSV 또는 JSONL 파일을 사용하세요.")
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 128
})

In [9]:
# 4. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 5. 양자화 설정 (선택적)
# 메모리 제약이 있는 경우 양자화를 사용할 수 있습니다
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [10]:

# 6. 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    # quantization_config=quantization_config, # instance에 따라서 양자화 지원 여부가 달라집니다. 
    device_map="auto",
    trust_remote_code=True,
)

# 7. LoRA 설정
# LoRA는 전체 모델을 미세 조정하는 대신 적은 수의 파라미터만 학습하여 효율적인 파인튜닝을 가능하게 합니다
peft_config = LoraConfig(
    r=16,                     # LoRA 행렬의 랭크
    lora_alpha=32,            # LoRA 알파 파라미터
    lora_dropout=0.05,        # LoRA 드롭아웃 비율
    bias="none",              # 바이어스 파라미터 학습 여부
    task_type="CAUSAL_LM",    # 태스크 유형
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # 타겟 모듈
)

# 8. 훈련 인자 설정
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    **training_params,
    remove_unused_columns=False,
    report_to="tensorboard",
    push_to_hub=False,  # True로 설정하면 Hugging Face Hub에 모델 업로드
)


In [11]:

# 9. 데이터 포맷 확인 및 처리 함수 정의
def formatting_func(example):
    # 데이터셋 형식에 따라 조정
    if "text" in example and "output" in example:
        # text, output 형식
        text = example["text"]
        output = example["output"]
        return f"{text} {output}"
    elif "instruction" in example and "response" in example:
        # instruction, response 형식
        instruction = example["instruction"]
        context = example.get("context", "")
        response = example["response"]

        if context:
            return f"<INST>{instruction}\n\n{context}</INST> {response}"
        else:
            return f"<INST>{instruction}</INST> {response}"
            
    elif "prompt" in example and "completion" in example:
        # instruction, response 형식
        instruction = example["prompt"]
        context = example.get("context", "")
        response = example["completion"]

        if context:
            return f"<INST>{instruction}\n\n{context}</INST> {response}"
        else:
            return f"<INST>{instruction}</INST> {response}"
            
    
    else:
        # 기본 형식 (단일 텍스트)
        return example["text"] if "text" in example else ""

# 10. SFT 트레이너 설정
trainer = SFTTrainer(
    model=model,
    # tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"] if "train" in dataset else dataset,
    
    peft_config=peft_config,
    formatting_func=formatting_func,
    # max_seq_length=max_seq_length,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [12]:
# 11. 모델 훈련
trainer.train()


Step,Training Loss
100,3.101100
200,1.332800
300,0.910600


TrainOutput(global_step=320, training_loss=1.7232888877391814, metrics={'train_runtime': 162.5483, 'train_samples_per_second': 7.875, 'train_steps_per_second': 1.969, 'total_flos': 172351956664320.0, 'train_loss': 1.7232888877391814})

In [13]:

# 12. 모델 저장
trainer.save_model(OUTPUT_DIR)

# 13. 토크나이저 저장
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"모델이 {OUTPUT_DIR}에 저장되었습니다.")

모델이 ./llama-3.2-1b-finetuned에 저장되었습니다.


# Inference

In [14]:
def test_model(question, model_path, tokenizer):
    """파인튜닝된 모델을 테스트합니다."""
    # 모델 로드
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )

    prompt = f"<INST>{question}</INST>"
    
    # 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.5,
            do_sample=True,
            top_p=0.9,
            # eos_token_id=128009,          # Llama 3.2 공식 EOS 토큰[1][3]
            # pad_token_id=128009,          # 패딩 토큰 통일[4]
            repetition_penalty=1.2,       # 반복 문구 억제[3]
        )

    # 결과 출력
    # generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("테스트 결과:")
    print(generated_text)

In [15]:
test_model("너 이름이 뭐니?.", OUTPUT_DIR, tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


테스트 결과:
<INST>너 이름이 뭐니?.</INST>저는 미니ai라는 이름을 가지고 있어요.


In [16]:
test_model("너에 대해서 소개 해. 이름이 뭐야?", OUTPUT_DIR, tokenizer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


테스트 결과:
<INST>너에 대해서 소개 해. 이름이 뭐야?</INST>저는 미니ai라는 이름을 가진 언어 모델입니다.</INST>

